In [1]:
# Importação da biblioteca pandas
import pandas as pd

In [2]:
# Instalação dos requisitos para o PySpark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [3]:
# Configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
# Torna o pyspark "importável"
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession

# Inicializar a SparkSession com suporte ao Hive
spark = SparkSession.builder \
    .appName("Spark with Hive on Colab") \
    .config("spark.sql.catalogImplementation", "hive") \
    .config("spark.sql.warehouse.dir", "/content/spark-warehouse") \
    .config("hive.metastore.warehouse.dir", "/content/spark-warehouse") \
    .enableHiveSupport() \
    .getOrCreate()

# Criar diretório para o warehouse
!mkdir -p /content/spark-warehouse

In [5]:
# Verifica o SparkContext
print(spark)

# Exibe a Spark version
print(spark.version)

3.1.1


In [6]:
cliente = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQW3fP3j4qoiMGBXDAGzg_9IW2b3zgjdkVKLsURNVe9QezpHXimWfKle_55CQQtkeWL69OAASBDNdk8/pub?gid=2073489257&single=true&output=csv')
display(cliente)

,codcliente,nome,cidade,sexo,estado,estadocivil
0,1,Ana Silva,Duque de Caxias,F,RJ,C
1,2,Bruna Pereira,Niterói,F,RJ,C
2,3,Túlio Nascimento,Duque de Caxias,M,RJ,S
3,4,Fernando Souza,Campinas,M,SP,S
4,5,Lúcia Andrade,São Paulo,F,SP,C


In [7]:
aluguel = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRncyLGO1iOo2H53JaryzVF1GPjUhWl9DsN7TZROCDxaP85iCwl5aW5ffBEzqtpAMRNYkd7eO5ehmgn/pub?gid=1581881382&single=true&output=csv')
display(aluguel)

,codaluguel,codcliente,codcarro,data_aluguel
0,1,3,2,2023-04-01
1,2,2,1,2023-04-02
2,3,2,1,2023-04-03
3,4,2,3,2023-04-04
4,5,1,4,2023-04-05
5,6,1,4,2023-04-13
6,7,1,1,2023-04-15
7,8,5,2,2023-04-19
8,9,5,2,2023-04-21
9,10,3,1,2023-04-25


In [8]:
carro = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSznlX7UXeH_LeKcNteiDnWvdwZyydAQl0_x8NU9cx6G00Zh7SMrjoUuNpytVq7U-iQVzQNJ7jC7GpY/pub?gid=306989914&single=true&output=csv')
display(carro)

,codcarro,codmarca,modelo,valor
0,1,1,Ka,100.0
1,2,2,Argo,150.0
2,3,3,Onix,170.0
3,4,4,Polo,150.0
4,5,5,Kwid,120.0


In [9]:
marca = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSI-4_QEFwZ6eDGwUDlip0_PGBn7d_F7j59UwXmRvWbQFyy01ENPatjkbO1E8k5ZW5lqSY9ox112j7X/pub?gid=1160143272&single=true&output=csv')
display(marca)

,codmarca,marca
0,1,Ford
1,2,Fiat
2,3,Chevrolet
3,4,Volkswagen
4,5,Renault


In [10]:
cliente.to_csv('cliente.csv',index=False)

In [11]:
aluguel.to_csv('aluguel.csv',index=False)

In [12]:
marca.to_csv('marca.csv',index=False)

In [13]:
carro.to_csv('carro.csv',index=False)

In [14]:
'''
01 - Carregar as quatro tabelas do banco locadora no PySpark como DataFrames
'''

aluguel_df = spark.read.csv("aluguel.csv", header=True, inferSchema=True)
carro_df = spark.read.csv("carro.csv", header=True, inferSchema=True)
cliente_df = spark.read.csv("cliente.csv", header=True, inferSchema=True)
marca_df = spark.read.csv("marca.csv", header=True, inferSchema=True)

In [15]:
'''
02 - Exibir as cinco primeiras linhas de cada DataFrame
'''

aluguel_df.show(5)
carro_df.show(5)
cliente_df.show(5)
marca_df.show(5)

+----------+----------+--------+------------+
|codaluguel|codcliente|codcarro|data_aluguel|
+----------+----------+--------+------------+
|         1|         3|       2|  2023-04-01|
|         2|         2|       1|  2023-04-02|
|         3|         2|       1|  2023-04-03|
|         4|         2|       3|  2023-04-04|
|         5|         1|       4|  2023-04-05|
+----------+----------+--------+------------+
only showing top 5 rows

+--------+--------+------+-----+
|codcarro|codmarca|modelo|valor|
+--------+--------+------+-----+
|       1|       1|    Ka|100.0|
|       2|       2|  Argo|150.0|
|       3|       3|  Onix|170.0|
|       4|       4|  Polo|150.0|
|       5|       5|  Kwid|120.0|
+--------+--------+------+-----+

+----------+----------------+---------------+----+------+-----------+
|codcliente|            nome|         cidade|sexo|estado|estadocivil|
+----------+----------------+---------------+----+------+-----------+
|         1|       Ana Silva|Duque de Caxias|   F|   

In [16]:
'''
03 - Contar o número de linhas e colunas de cada tabela
'''

print(f"Aluguel: {aluguel_df.count()} linhas, {len(aluguel_df.columns)} colunas")
print(f"Carro: {carro_df.count()} linhas, {len(carro_df.columns)} colunas")
print(f"Cliente: {cliente_df.count()} linhas, {len(cliente_df.columns)} colunas")
print(f"Marca: {marca_df.count()} linhas, {len(marca_df.columns)} colunas")

Aluguel: 10 linhas, 4 colunas
Carro: 5 linhas, 4 colunas
Cliente: 5 linhas, 6 colunas
Marca: 5 linhas, 2 colunas


In [17]:
'''
04 - Exibir o esquema (schema) de cada DataFrame
'''

aluguel_df.printSchema()
carro_df.printSchema()
cliente_df.printSchema()
marca_df.printSchema()

root
 |-- codaluguel: integer (nullable = true)
 |-- codcliente: integer (nullable = true)
 |-- codcarro: integer (nullable = true)
 |-- data_aluguel: string (nullable = true)

root
 |-- codcarro: integer (nullable = true)
 |-- codmarca: integer (nullable = true)
 |-- modelo: string (nullable = true)
 |-- valor: double (nullable = true)

root
 |-- codcliente: integer (nullable = true)
 |-- nome: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- estadocivil: string (nullable = true)

root
 |-- codmarca: integer (nullable = true)
 |-- marca: string (nullable = true)



In [24]:
'''
05 - Renomear as colunas dos DataFrames para ter nomes mais amigáveis
'''
aluguel_df = aluguel_df.withColumnRenamed("id", "aluguel_id")
cliente_df = cliente_df.withColumnRenamed("id", "cliente_id")
carro_df = carro_df.withColumnRenamed("id", "carro_id")
marca_df = marca_df.withColumnRenamed("id", "marca_id")

In [29]:
'''
06 - Selecionar apenas os aluguéis realizados após uma data específica
'''
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Criar a sessão do Spark
spark = SparkSession.builder.appName("Locadora").getOrCreate()

# Criar um DataFrame de exemplo com os dados desejados
data = [
    (7, 1, 1, "2023-04-15"),
    (8, 5, 2, "2023-04-19"),
    (9, 5, 2, "2023-04-21"),
    (10, 3, 1, "2023-04-25")
]

columns = ["codaluguel", "codcliente", "codcarro", "data_aluguel"]

# Criar o DataFrame
aluguel = spark.createDataFrame(data, columns)

# Definir uma data específica
data_especifica = "2023-04-13"

# Filtrar aluguéis realizados após a data específica
aluguel_filtrado = aluguel.where(F.col("data_aluguel") > data_especifica)

# Exibir os resultados
aluguel_filtrado.show()

+----------+----------+--------+------------+
|codaluguel|codcliente|codcarro|data_aluguel|
+----------+----------+--------+------------+
|         7|         1|       1|  2023-04-15|
|         8|         5|       2|  2023-04-19|
|         9|         5|       2|  2023-04-21|
|        10|         3|       1|  2023-04-25|
+----------+----------+--------+------------+



In [ ]:
'''
07 - Encontrar clientes que residem no estado de "RJ"
'''

clientes_rj = cliente_df.filter(col("estado_cliente") == "RJ")
clientes_rj.show()

+----------+----------------+---------------+----+--------------+-----------+
|codcliente|            nome|         cidade|sexo|estado_cliente|estadocivil|
+----------+----------------+---------------+----+--------------+-----------+
|         1|       Ana Silva|Duque de Caxias|   F|            RJ|          C|
|         2|   Bruna Pereira|        Niterói|   F|            RJ|          C|
|         3|Túlio Nascimento|Duque de Caxias|   M|            RJ|          S|
+----------+----------------+---------------+----+--------------+-----------+



In [ ]:
'''
08 - Filtrar carros com valor de aluguel maior que 150
'''

carros_caros = carro_df.filter(col("valor_aluguel") > 150)
carros_caros.show()

+--------+--------+------+-------------+
|codcarro|codmarca|modelo|valor_aluguel|
+--------+--------+------+-------------+
|       3|       3|  Onix|        170.0|
+--------+--------+------+-------------+



In [ ]:
'''
09 - Selecionar aluguéis onde o cliente é do sexo feminino
'''
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# Inicializando a SparkSession
spark = SparkSession.builder \
    .appName("Locadora") \
    .getOrCreate()

# Carregando os DataFrames (exemplo)
aluguel = spark.read.csv("aluguel.csv", header=True, inferSchema=True)
cliente = spark.read.csv("cliente.csv", header=True, inferSchema=True)

# Filtrando aluguéis onde o cliente é do sexo feminino
alugueis_femininos = aluguel.join(cliente, aluguel.codcliente == cliente.codcliente) \
                             .where(cliente.sexo == 'F') \
                             .select(aluguel.codaluguel, cliente.nome, cliente.sexo)

# Exibindo os resultados
alugueis_femininos.show()

# Fechando a SparkSession
spark.stop()

+----------+-------------+----+
|codaluguel|         nome|sexo|
+----------+-------------+----+
|         2|Bruna Pereira|   F|
|         3|Bruna Pereira|   F|
|         4|Bruna Pereira|   F|
|         5|    Ana Silva|   F|
|         6|    Ana Silva|   F|
|         7|    Ana Silva|   F|
|         8|Lúcia Andrade|   F|
|         9|Lúcia Andrade|   F|
+----------+-------------+----+



In [ ]:
'''
10 - Identificar clientes solteiros
'''
from pyspark.sql import SparkSession
import pyspark.sql.functions as F  # Importando funções como F

# Inicializando a SparkSession
spark = SparkSession.builder \
    .appName("Locadora") \
    .getOrCreate()

# Carregando o DataFrame de Clientes (exemplo)
cliente = spark.read.csv("cliente.csv", header=True, inferSchema=True)

# Filtrando clientes solteiros
clientes_solteiros = cliente.where(F.col("estadocivil") == "S")

# Exibindo os resultados
clientes_solteiros.show()

# Fechando a SparkSession
spark.stop()

+----------+----------------+---------------+----+------+-----------+
|codcliente|            nome|         cidade|sexo|estado|estadocivil|
+----------+----------------+---------------+----+------+-----------+
|         3|Túlio Nascimento|Duque de Caxias|   M|    RJ|          S|
|         4|  Fernando Souza|       Campinas|   M|    SP|          S|
+----------+----------------+---------------+----+------+-----------+



In [ ]:
'''
11 - Realizar um join entre "Aluguel" e "Cliente"
'''
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# Inicializando a SparkSession
spark = SparkSession.builder \
    .appName("Locadora") \
    .getOrCreate()

# Carregando os DataFrames (exemplo)
aluguel_df = spark.read.csv("aluguel.csv", header=True, inferSchema=True)
cliente_df = spark.read.csv("cliente.csv", header=True, inferSchema=True)

# Verificando os nomes das colunas
print("Colunas do DataFrame de Aluguel:")
print(aluguel_df.columns)

print("Colunas do DataFrame de Cliente:")
print(cliente_df.columns)

# Realizando o join entre Aluguel e Cliente
aluguel_cliente = aluguel_df.join(cliente_df, aluguel_df.codcliente == cliente_df.codcliente, "inner")

# Exibindo os resultados
aluguel_cliente.show()

# Fechando a SparkSession
spark.stop()

Colunas do DataFrame de Aluguel:
['codaluguel', 'codcliente', 'codcarro', 'data_aluguel']
Colunas do DataFrame de Cliente:
['codcliente', 'nome', 'cidade', 'sexo', 'estado', 'estadocivil']
+----------+----------+--------+------------+----------+----------------+---------------+----+------+-----------+
|codaluguel|codcliente|codcarro|data_aluguel|codcliente|            nome|         cidade|sexo|estado|estadocivil|
+----------+----------+--------+------------+----------+----------------+---------------+----+------+-----------+
|         7|         1|       1|  2023-04-15|         1|       Ana Silva|Duque de Caxias|   F|    RJ|          C|
|         6|         1|       4|  2023-04-13|         1|       Ana Silva|Duque de Caxias|   F|    RJ|          C|
|         5|         1|       4|  2023-04-05|         1|       Ana Silva|Duque de Caxias|   F|    RJ|          C|
|         4|         2|       3|  2023-04-04|         2|   Bruna Pereira|        Niterói|   F|    RJ|          C|
|         3| 

In [31]:
'''
12 - Juntar "Carro" e "Marca"
'''
from pyspark.sql import SparkSession

# Criar a sessão do Spark
spark = SparkSession.builder.appName("Locadora").getOrCreate()

# Criar DataFrame de exemplo para carros
carro_data = [
    (1, 1, "Ka", 100.0),  # codcarro, codmarca, modelo, valor
    (2, 2, "Argo", 150.0),
    (3, 3, "Onix", 170.0),
    (4, 4, "Polo", 150.0),
    (5, 5, "Kwid", 120.0)
]

carro_columns = ["codcarro", "codmarca", "modelo", "valor"]

# Criar DataFrame de exemplo para marcas
marca_data = [
    (1, "Ford"),        # codmarca, marca
    (2, "Fiat"),
    (3, "Chevrolet"),
    (4, "Volkswagen"),
    (5, "Renault")
]

marca_columns = ["codmarca", "marca"]

# Criar DataFrames
carro_df = spark.createDataFrame(carro_data, carro_columns)
marca_df = spark.createDataFrame(marca_data, marca_columns)

# Juntar "Carro" e "Marca" usando a coluna codmarca
carro_marca_df = carro_df.join(marca_df, "codmarca")

# Exibir o resultado da junção
carro_marca_df.show()

+--------+--------+------+-----+----------+
|codmarca|codcarro|modelo|valor|     marca|
+--------+--------+------+-----+----------+
|       5|       5|  Kwid|120.0|   Renault|
|       1|       1|    Ka|100.0|      Ford|
|       3|       3|  Onix|170.0| Chevrolet|
|       2|       2|  Argo|150.0|      Fiat|
|       4|       4|  Polo|150.0|Volkswagen|
+--------+--------+------+-----+----------+



In [36]:
'''
13 - Criar um DataFrame combinando "Aluguel", "Carro" e "Cliente"
'''

from pyspark.sql import SparkSession

# Criar a sessão do Spark
spark = SparkSession.builder.appName("Locadora").getOrCreate()

# Criar DataFrame para aluguéis
aluguel_data = [
    (7, 1, 1, "2023-04-15"),
    (6, 1, 4, "2023-04-13"),
    (5, 1, 4, "2023-04-05"),
    (4, 2, 3, "2023-04-04"),
    (3, 2, 1, "2023-04-03"),
    (2, 2, 1, "2023-04-02"),
    (10, 3, 1, "2023-04-25"),
    (1, 3, 2, "2023-04-01"),
    (9, 5, 2, "2023-04-21"),
    (8, 5, 2, "2023-04-19")
]
aluguel_columns = ["codaluguel", "codcliente", "codcarro", "data_aluguel"]

# Criar DataFrame para clientes
cliente_data = [
    (1, "Ana Silva", "Duque de Caxias", "F", "RJ", "C"),
    (2, "Bruna Pereira", "Niterói", "F", "RJ", "C"),
    (3, "Túlio Nascimento", "Duque de Caxias", "M", "RJ", "S"),
    (5, "Lúcia Andrade", "São Paulo", "F", "SP", "C")
]
cliente_columns = ["codcliente", "nome", "cidade", "sexo", "estado", "estadocivil"]

# Criar DataFrame para carros
carro_data = [
    (1, 1, "Ka", 100.0),
    (2, 2, "Argo", 150.0),
    (3, 3, "Onix", 170.0),
    (4, 4, "Polo", 150.0),
    (5, 5, "Kwid", 120.0)
]
carro_columns = ["codcarro", "codmarca", "modelo", "valor"]

# Criar DataFrame para marcas
marca_data = [
    (1, "Ford"),
    (2, "Fiat"),
    (3, "Chevrolet"),
    (4, "Volkswagen"),
    (5, "Renault")
]
marca_columns = ["codmarca", "marca"]

# Criar os DataFrames
aluguel_df = spark.createDataFrame(aluguel_data, aluguel_columns)
cliente_df = spark.createDataFrame(cliente_data, cliente_columns)
carro_df = spark.createDataFrame(carro_data, carro_columns)
marca_df = spark.createDataFrame(marca_data, marca_columns)

# Junção dos DataFrames com aliases
resultado = aluguel_df.alias("a") \
    .join(cliente_df.alias("c"), "codcliente") \
    .join(carro_df.alias("car"), "codcarro") \
    .join(marca_df.alias("m"), carro_df.codmarca == marca_df.codmarca) \
    .select(
        "a.codaluguel", "a.codcliente", "a.codcarro", "a.data_aluguel",
        "c.nome", "c.cidade", "c.sexo", "c.estado", "c.estadocivil",
        "car.modelo", "car.valor", "m.marca"
    )

# Exibir o resultado
resultado.show(truncate=False)

+----------+----------+--------+------------+----------------+---------------+----+------+-----------+------+-----+----------+
|codaluguel|codcliente|codcarro|data_aluguel|nome            |cidade         |sexo|estado|estadocivil|modelo|valor|marca     |
+----------+----------+--------+------------+----------------+---------------+----+------+-----------+------+-----+----------+
|7         |1         |1       |2023-04-15  |Ana Silva       |Duque de Caxias|F   |RJ    |C          |Ka    |100.0|Ford      |
|10        |3         |1       |2023-04-25  |Túlio Nascimento|Duque de Caxias|M   |RJ    |S          |Ka    |100.0|Ford      |
|3         |2         |1       |2023-04-03  |Bruna Pereira   |Niterói        |F   |RJ    |C          |Ka    |100.0|Ford      |
|2         |2         |1       |2023-04-02  |Bruna Pereira   |Niterói        |F   |RJ    |C          |Ka    |100.0|Ford      |
|4         |2         |3       |2023-04-04  |Bruna Pereira   |Niterói        |F   |RJ    |C          |Onix  |17

In [38]:
'''
14 - Realizar um join entre "Cliente" e "Carro" com uma condição específica
'''
from pyspark.sql import SparkSession

# Criar a sessão do Spark
spark = SparkSession.builder.appName("Locadora").getOrCreate()

# Criar DataFrame para clientes
cliente_data = [
    (1, "Ana Silva", "Duque de Caxias", "F", "RJ", "C"),
    (2, "Bruna Pereira", "Niterói", "F", "RJ", "C"),
    (3, "Túlio Nascimento", "Duque de Caxias", "M", "RJ", "S"),
    (5, "Lúcia Andrade", "São Paulo", "F", "SP", "C")
]
cliente_columns = ["codcliente", "nome", "cidade", "sexo", "estado", "estadocivil"]

# Criar DataFrame para carros
carro_data = [
    (1, 1, "Ka", 100.0),
    (2, 2, "Argo", 150.0),
    (3, 3, "Onix", 170.0),
    (4, 4, "Polo", 150.0),
    (5, 5, "Kwid", 120.0)
]
carro_columns = ["codcarro", "codmarca", "modelo", "valor"]

# Criar DataFrame para aluguéis (relacionando clientes e carros)
aluguel_data = [
    (1, 1, 1),  # (codaluguel, codcliente, codcarro)
    (2, 2, 2),
    (3, 3, 3),
    (4, 1, 4),
    (5, 5, 5)
]
aluguel_columns = ["codaluguel", "codcliente", "codcarro"]

# Criar os DataFrames
cliente_df = spark.createDataFrame(cliente_data, cliente_columns)
carro_df = spark.createDataFrame(carro_data, carro_columns)
aluguel_df = spark.createDataFrame(aluguel_data, aluguel_columns)

# Realizar o join entre Cliente e Carro através do Aluguel
resultado = aluguel_df.alias("a") \
    .join(cliente_df.alias("c"), "codcliente") \
    .join(carro_df.alias("car"), "codcarro") \
    .filter(carro_df.valor > 120.0) \
    .select("c.nome", "c.cidade", "c.sexo", "car.modelo", "car.valor")

# Exibir o resultado
resultado.show(truncate=False)

+----------------+---------------+----+------+-----+
|nome            |cidade         |sexo|modelo|valor|
+----------------+---------------+----+------+-----+
|Túlio Nascimento|Duque de Caxias|M   |Onix  |170.0|
|Bruna Pereira   |Niterói        |F   |Argo  |150.0|
|Ana Silva       |Duque de Caxias|F   |Polo  |150.0|
+----------------+---------------+----+------+-----+



In [40]:
'''
15 - Encontrar o valor médio dos carros alugados
'''
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Criar a sessão do Spark
spark = SparkSession.builder.appName("Locadora").getOrCreate()

# Criar DataFrame para carros
carro_data = [
    (1, 1, "Ka", 100.0),
    (2, 2, "Argo", 150.0),
    (3, 3, "Onix", 170.0),
    (4, 4, "Polo", 150.0),
    (5, 5, "Kwid", 120.0)
]
carro_columns = ["codcarro", "codmarca", "modelo", "valor"]

# Criar DataFrame para aluguéis (relacionando clientes e carros)
aluguel_data = [
    (1, 1, 1),  # (codaluguel, codcliente, codcarro)
    (2, 2, 2),
    (3, 3, 3),
    (4, 1, 4),
    (5, 5, 5)
]
aluguel_columns = ["codaluguel", "codcliente", "codcarro"]

# Criar os DataFrames
carro_df = spark.createDataFrame(carro_data, carro_columns)
aluguel_df = spark.createDataFrame(aluguel_data, aluguel_columns)

# Realizar o join entre Aluguel e Carro para calcular o valor médio
resultado = aluguel_df.alias("a") \
    .join(carro_df.alias("car"), "codcarro") \
    .agg(F.avg("car.valor").alias("valor_media"))

# Exibir o resultado
resultado.show(truncate=False)

+-----------+
|valor_media|
+-----------+
|138.0      |
+-----------+



In [41]:
'''
16 - Calcular o número total de clientes por estado
'''
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Criar a sessão do Spark
spark = SparkSession.builder.appName("Locadora").getOrCreate()

# Criar DataFrame para clientes
cliente_data = [
    (1, "Ana Silva", "Duque de Caxias", "F", "RJ", "C"),
    (2, "Bruna Pereira", "Niterói", "F", "RJ", "C"),
    (3, "Túlio Nascimento", "Duque de Caxias", "M", "RJ", "S"),
    (4, "Lúcia Andrade", "São Paulo", "F", "SP", "C"),
    (5, "Carlos Alberto", "São Paulo", "M", "SP", "C")
]
cliente_columns = ["codcliente", "nome", "cidade", "sexo", "estado", "estadocivil"]

# Criar o DataFrame
cliente_df = spark.createDataFrame(cliente_data, cliente_columns)

# Calcular o número total de clientes por estado
resultado = cliente_df.groupBy("estado").agg(F.count("codcliente").alias("count"))

# Exibir o resultado
resultado.show(truncate=False)

+------+-----+
|estado|count|
+------+-----+
|SP    |2    |
|RJ    |3    |
+------+-----+



In [42]:
'''
17 - Identificar a marca mais popular com base nos aluguéis
'''
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Criar a sessão do Spark
spark = SparkSession.builder.appName("Locadora").getOrCreate()

# Criar DataFrame para marcas
marca_data = [
    (1, "Ford"),
    (2, "Fiat"),
    (3, "Chevrolet"),
    (4, "Volkswagen"),
    (5, "Renault")
]
marca_columns = ["codmarca", "marca"]

# Criar DataFrame para carros
carro_data = [
    (1, 1, "Ka", 100.0),
    (2, 2, "Argo", 150.0),
    (3, 3, "Onix", 170.0),
    (4, 4, "Polo", 150.0),
    (5, 5, "Kwid", 120.0)
]
carro_columns = ["codcarro", "codmarca", "modelo", "valor"]

# Criar DataFrame para aluguéis (relacionando clientes e carros)
aluguel_data = [
    (1, 1, 1),  # (codaluguel, codcliente, codcarro)
    (2, 2, 2),
    (3, 3, 3),
    (4, 1, 4),
    (5, 5, 5),
    (6, 1, 1),
    (7, 2, 2),
    (8, 3, 3),
    (9, 4, 4),
    (10, 5, 5)
]
aluguel_columns = ["codaluguel", "codcliente", "codcarro"]

# Criar os DataFrames
marca_df = spark.createDataFrame(marca_data, marca_columns)
carro_df = spark.createDataFrame(carro_data, carro_columns)
aluguel_df = spark.createDataFrame(aluguel_data, aluguel_columns)

# Realizar o join entre Aluguel, Carro e Marca
resultado = aluguel_df.alias("a") \
    .join(carro_df.alias("car"), "codcarro") \
    .join(marca_df.alias("m"), carro_df.codmarca == marca_df.codmarca) \
    .groupBy("m.marca") \
    .agg(F.count("a.codaluguel").alias("qtde_alugueis")) \
    .orderBy(F.desc("qtde_alugueis")) \
    .limit(1)

# Exibir o resultado
resultado.show(truncate=False)

+----------+-------------+
|marca     |qtde_alugueis|
+----------+-------------+
|Volkswagen|2            |
+----------+-------------+



In [43]:
'''
18 - Determinar o maior e menor valor de aluguel entre os carros
'''
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Criar a sessão do Spark
spark = SparkSession.builder.appName("Locadora").getOrCreate()

# Criar DataFrame para carros
carro_data = [
    (1, 1, "Ka", 100.0),
    (2, 2, "Argo", 150.0),
    (3, 3, "Onix", 170.0),
    (4, 4, "Polo", 150.0),
    (5, 5, "Kwid", 120.0)
]
carro_columns = ["codcarro", "codmarca", "modelo", "valor"]

# Criar o DataFrame
carro_df = spark.createDataFrame(carro_data, carro_columns)

# Calcular o maior e menor valor de aluguel
resultado = carro_df.agg(
    F.max("valor").alias("maior_valor"),
    F.min("valor").alias("menor_valor")
)

# Exibir o resultado
resultado.show(truncate=False)

+-----------+-----------+
|maior_valor|menor_valor|
+-----------+-----------+
|170.0      |100.0      |
+-----------+-----------+



In [44]:
'''
19 - Classificar os carros pelo valor do aluguel em ordem decrescente
'''
from pyspark.sql import SparkSession

# Criar a sessão do Spark
spark = SparkSession.builder.appName("Locadora").getOrCreate()

# Criar DataFrame para carros
carro_data = [
    (1, 1, "Ka", 100.0),
    (2, 2, "Argo", 150.0),
    (3, 3, "Onix", 170.0),
    (4, 4, "Polo", 150.0),
    (5, 5, "Kwid", 120.0)
]
carro_columns = ["codcarro", "codmarca", "modelo", "valor"]

# Criar o DataFrame
carro_df = spark.createDataFrame(carro_data, carro_columns)

# Classificar os carros pelo valor do aluguel em ordem decrescente
resultado = carro_df.orderBy("valor", ascending=False)

# Exibir o resultado
resultado.show(truncate=False)

+--------+--------+------+-----+
|codcarro|codmarca|modelo|valor|
+--------+--------+------+-----+
|3       |3       |Onix  |170.0|
|2       |2       |Argo  |150.0|
|4       |4       |Polo  |150.0|
|5       |5       |Kwid  |120.0|
|1       |1       |Ka    |100.0|
+--------+--------+------+-----+



In [45]:
'''
20 - Calcular a diferença em dias entre o aluguel mais recente e o mais antigo
'''
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Criar a sessão do Spark
spark = SparkSession.builder.appName("Locadora").getOrCreate()

# Criar DataFrame para aluguéis (com datas)
aluguel_data = [
    (1, 1, "2023-01-01"),  # (codaluguel, codcliente, data_aluguel)
    (2, 2, "2023-01-10"),
    (3, 3, "2023-01-15"),
    (4, 1, "2023-01-25"),
    (5, 5, "2023-01-05")
]
aluguel_columns = ["codaluguel", "codcliente", "data_aluguel"]

# Criar o DataFrame
aluguel_df = spark.createDataFrame(aluguel_data, aluguel_columns)

# Converter a coluna de data para o tipo Date
aluguel_df = aluguel_df.withColumn("data_aluguel", F.to_date("data_aluguel"))

# Encontrar a data mais recente e a mais antiga
datas = aluguel_df.agg(
    F.min("data_aluguel").alias("data_antiga"),
    F.max("data_aluguel").alias("data_recente")
)

# Calcular a diferença em dias
resultado = datas.select(
    (F.datediff(F.col("data_recente"), F.col("data_antiga"))).alias("diferenca_dias")
)

# Exibir o resultado
resultado.show(truncate=False)

# Exibir a diferença em dias
diferenca = resultado.collect()[0]["diferenca_dias"]
print(f"Diferença em dias entre o aluguel mais recente e o mais antigo: {diferenca} dias")

+--------------+
|diferenca_dias|
+--------------+
|24            |
+--------------+

Diferença em dias entre o aluguel mais recente e o mais antigo: 24 dias


In [46]:
'''
21 - Criar uma coluna no DataFrame "Carro" para categorizar os valores de aluguel
'''
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Criar a sessão do Spark
spark = SparkSession.builder.appName("Locadora").getOrCreate()

# Criar DataFrame para carros
carro_data = [
    (1, 1, "Ka", 100.0),
    (2, 2, "Argo", 150.0),
    (3, 3, "Onix", 170.0),
    (4, 4, "Polo", 150.0),
    (5, 5, "Kwid", 120.0)
]
carro_columns = ["codcarro", "codmarca", "modelo", "valor"]

# Criar o DataFrame
carro_df = spark.createDataFrame(carro_data, carro_columns)

# Criar a coluna categoria_valor com base nos valores
carro_df = carro_df.withColumn(
    "categoria_valor",
    F.when(carro_df.valor < 120, "Baixo")
     .when((carro_df.valor >= 120) & (carro_df.valor <= 150), "Médio")
     .otherwise("Alto")
)

# Exibir o resultado
carro_df.show(truncate=False)

+--------+--------+------+-----+---------------+
|codcarro|codmarca|modelo|valor|categoria_valor|
+--------+--------+------+-----+---------------+
|1       |1       |Ka    |100.0|Baixo          |
|2       |2       |Argo  |150.0|Médio          |
|3       |3       |Onix  |170.0|Alto           |
|4       |4       |Polo  |150.0|Médio          |
|5       |5       |Kwid  |120.0|Médio          |
+--------+--------+------+-----+---------------+



In [47]:
'''
22 - Criar uma coluna no DataFrame "Cliente" para indicar se a cidade é a capital do estado
'''

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Criar a sessão do Spark
spark = SparkSession.builder.appName("Locadora").getOrCreate()

# Criar DataFrame para clientes
cliente_data = [
    (1, "Ana Silva", "Duque de Caxias", "F", "RJ", "C"),
    (2, "Bruna Pereira", "Niterói", "F", "RJ", "C"),
    (3, "Túlio Nascimento", "Duque de Caxias", "M", "RJ", "S"),
    (4, "Fernando Souza", "Campinas", "M", "SP", "S"),
    (5, "Lúcia Andrade", "São Paulo", "F", "SP", "C")
]
cliente_columns = ["codcliente", "nome", "cidade", "sexo", "estado", "estadocivil"]

# Criar o DataFrame
cliente_df = spark.createDataFrame(cliente_data, cliente_columns)

# Definir as capitais dos estados
capitais = {
    "RJ": "Rio de Janeiro",
    "SP": "São Paulo"
}

# Criar a coluna 'cidade_capital'
cliente_df = cliente_df.withColumn(
    "cidade_capital",
    F.when(cliente_df.cidade.isin(*capitais.values()), "Sim")
     .otherwise("Interior")
)

# Exibir o resultado
cliente_df.show(truncate=False)

+----------+----------------+---------------+----+------+-----------+--------------+
|codcliente|nome            |cidade         |sexo|estado|estadocivil|cidade_capital|
+----------+----------------+---------------+----+------+-----------+--------------+
|1         |Ana Silva       |Duque de Caxias|F   |RJ    |C          |Interior      |
|2         |Bruna Pereira   |Niterói        |F   |RJ    |C          |Interior      |
|3         |Túlio Nascimento|Duque de Caxias|M   |RJ    |S          |Interior      |
|4         |Fernando Souza  |Campinas       |M   |SP    |S          |Interior      |
|5         |Lúcia Andrade   |São Paulo      |F   |SP    |C          |Sim           |
+----------+----------------+---------------+----+------+-----------+--------------+



In [48]:
'''
23 - Adicionar uma coluna em "Aluguel" com o valor total do aluguel, considerando uma taxa fixa de 10%
'''
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Criar a sessão do Spark
spark = SparkSession.builder.appName("Locadora").getOrCreate()

# Criar DataFrame para carros
carro_data = [
    (1, 1, "Ka", 100.0),
    (2, 2, "Argo", 150.0),
    (3, 3, "Onix", 170.0),
    (4, 4, "Polo", 150.0),
    (5, 5, "Kwid", 120.0)
]
carro_columns = ["codcarro", "codmarca", "modelo", "valor"]

# Criar DataFrame para aluguéis (com o modelo do carro)
aluguel_data = [
    (1, 1, 1),  # (codaluguel, codcliente, codcarro)
    (2, 2, 1),
    (3, 3, 1),
    (4, 4, 1),
    (5, 5, 2),
    (6, 2, 2),
    (7, 3, 2),
    (8, 1, 3),
    (9, 4, 4),
    (10, 5, 4)
]
aluguel_columns = ["codaluguel", "codcliente", "codcarro"]

# Criar os DataFrames
carro_df = spark.createDataFrame(carro_data, carro_columns)
aluguel_df = spark.createDataFrame(aluguel_data, aluguel_columns)

# Realizar o join entre Aluguel e Carro
resultado = aluguel_df.alias("a") \
    .join(carro_df.alias("car"), "codcarro") \
    .select("car.modelo", "car.valor") \
    .withColumn("valor_total", F.col("valor") * 1.1)  # Calcula 10% a mais

# Exibir o resultado
resultado.show(truncate=False)

+------+-----+------------------+
|modelo|valor|valor_total       |
+------+-----+------------------+
|Ka    |100.0|110.00000000000001|
|Ka    |100.0|110.00000000000001|
|Ka    |100.0|110.00000000000001|
|Ka    |100.0|110.00000000000001|
|Onix  |170.0|187.00000000000003|
|Argo  |150.0|165.0             |
|Argo  |150.0|165.0             |
|Argo  |150.0|165.0             |
|Polo  |150.0|165.0             |
|Polo  |150.0|165.0             |
+------+-----+------------------+



In [49]:
'''
24 - Agrupar os aluguéis por cliente e contar o número de carros alugados
'''

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Criar a sessão do Spark
spark = SparkSession.builder.appName("Locadora").getOrCreate()

# Criar DataFrame para clientes
cliente_data = [
    (1, "Ana Silva", "Duque de Caxias", "F", "RJ", "C"),
    (2, "Bruna Pereira", "Niterói", "F", "RJ", "C"),
    (3, "Túlio Nascimento", "Duque de Caxias", "M", "RJ", "S"),
    (4, "Fernando Souza", "Campinas", "M", "SP", "S"),
    (5, "Lúcia Andrade", "São Paulo", "F", "SP", "C")
]
cliente_columns = ["codcliente", "nome", "cidade", "sexo", "estado", "estadocivil"]

# Criar DataFrame para aluguéis (com o modelo do carro)
aluguel_data = [
    (1, 1, 1),  # (codaluguel, codcliente, codcarro)
    (2, 1, 2),
    (3, 1, 3),
    (4, 2, 1),
    (5, 2, 2),
    (6, 2, 3),
    (7, 3, 1),
    (8, 4, 2),
    (9, 5, 3),
    (10, 5, 4)
]
aluguel_columns = ["codaluguel", "codcliente", "codcarro"]

# Criar os DataFrames
cliente_df = spark.createDataFrame(cliente_data, cliente_columns)
aluguel_df = spark.createDataFrame(aluguel_data, aluguel_columns)

# Realizar o join entre Aluguel e Cliente
resultado = aluguel_df.alias("a") \
    .join(cliente_df.alias("c"), "codcliente") \
    .groupBy("c.nome") \
    .agg(F.count("a.codaluguel").alias("qtde_alugueis")) \
    .orderBy("qtde_alugueis", ascending=False)

# Exibir o resultado
resultado.show(truncate=False)

+----------------+-------------+
|nome            |qtde_alugueis|
+----------------+-------------+
|Bruna Pereira   |3            |
|Ana Silva       |3            |
|Lúcia Andrade   |2            |
|Túlio Nascimento|1            |
|Fernando Souza  |1            |
+----------------+-------------+



In [50]:
'''
25 - Criar um script PySpark para agendar a execução automática das transformações
'''
from pyspark.sql import SparkSession

# Criar a sessão do Spark
spark = SparkSession.builder \
    .appName("Transformacao Carro") \
    .getOrCreate()

# Carregar os dados
carro = spark.read.csv("carro.csv", header=True, inferSchema=True)

# Realizar transformação simples
carro_transformado = carro.withColumn("valor_corrigido", carro["valor"] * 1.10)

# Salvar resultado
carro_transformado.write.csv("aluguel_transformado.csv", header=True)

# Encerrar a sessão do Spark
spark.stop()